# Spell Detection

## Introduction

Consider these symbols:

<img src="https://i.pinimg.com/originals/c8/63/bc/c863bc6b913dbc49e094ee20eddec0ed.jpg" alt="Drawing" style="width: 300px;"/>

When drawing with the PiNoir and using a reflective wand, they pretty much look like this:

<img src="../../media/capture.png" style="width: 200px;">

Not unlike a MNIST. Now, considering the fact that I had to draw the symbols myself, and even with the previous [augmenting](./corpus_augment.ipynb), let's create a really simple net so far.

## The Model

In [10]:
from keras.layers import Dense, Activation, Reshape, Input
from keras.layers import Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import os

In [2]:
NUMBER_OF_EPOCHS=50
IMAGE_SHAPE=(32, 32, 1)

In [3]:
def spell_net(number_of_classess=6, input_shape = (32, 32, 3)):
    """
    Returns a single convoluted net for MNIST given a number_of_classes and an input shape
    """
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='valid', name='conv1', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', use_bias=False)(x)
    x = BatchNormalization()(x)
    predictions = Dense(number_of_classess, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    print("spell net created")
    print(model.summary())
    return model

In [4]:
BATCH_SIZE = 64
NUM_EPOCH = 50
LR = 0.0002  # initial learning rate
B1 = 0.5  # momentum term
GENERATED_MODEL_PATH = '../../wand/spell_net/models/'

In [5]:
train_datagen = ImageDataGenerator(
        rescale=1./127.5,
        shear_range=0.052,
        zoom_range=0.05)
test_datagen = ImageDataGenerator(rescale=1./127.5)

train_generator = train_datagen.flow_from_directory(
        '../../media/dataset/train',
        target_size=(32, 32),
        batch_size=64)
test_generator = train_datagen.flow_from_directory(
        '../../media/dataset/test',
        target_size=(32, 32),
        batch_size=BATCH_SIZE)

Found 48000 images belonging to 6 classes.
Found 12000 images belonging to 6 classes.


In [6]:
if not os.path.exists(GENERATED_MODEL_PATH):
    os.makedirs(GENERATED_MODEL_PATH)

In [11]:
train_callbacks = [
    EarlyStopping(monitor='val_loss', patience=25, verbose=0, mode='min'),
    ModelCheckpoint(filepath=os.path.join(GENERATED_MODEL_PATH, "spell_net_{epoch:02d}.h5"),
                    verbose=1),
    TensorBoard(log_dir=GENERATED_MODEL_PATH, write_graph=False),   
]
s = spell_net(number_of_classess=test_generator.num_classes)
s.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
s.fit_generator(
            train_generator,
            steps_per_epoch=(len(train_generator.filenames)) // train_generator.batch_size,
            epochs=NUM_EPOCH,
            validation_data=test_generator,
            validation_steps=(len(test_generator.filenames)) // test_generator.batch_size,
            callbacks=train_callbacks,
            verbose=1
        )

spell net created
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 15, 15, 32)        896       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 32)          128       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               200704    
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512

Finally, our network is trained. 
The latest model can be found [here](https://s3.amazonaws.com/pipotter/spell_net/spell_net.tar.gz)